In [1]:
import pandas as pd
import numpy as np
from astropy.time import Time
from math import radians, cos, sin, asin, sqrt
import re
import raadpy as rp

Warning! Cupy not found! GPU accelleration is not available



In [2]:
def haversine(lon1, lat1, lon2, lat2):  #Calculate the great circle distance between two points on the earth 
                                        #(specified in decimal degrees)
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2]) # convert decimal degrees to radians 
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles.
    return c * r

In [3]:
# File of locations of all satellites. Location points per five minutes.
Location_File_All_Sats = "../../Data/Location/ISS_FERMI_Light-1_Locations.csv"
Locations_data_pd = pd.read_csv(Location_File_All_Sats)
Locations_headers = list(Locations_data_pd.keys())
Locations_data    = [i for i in Locations_data_pd.to_numpy().T]
Locations_data    = dict(zip(Locations_headers,Locations_data))

# Distance between all sats using great circle distance:
distance_ISS_Light1 = [haversine(Locations_data['Lon(deg)Light-1'][i],Locations_data['Lat(deg)Light-1'][i],Locations_data['Lon(deg)ISS'][i],Locations_data['Lat(deg)ISS'][i]) for i in range(len(Locations_data['Time(ModJDate)']))]
distance_ISS_Light1 = np.array(distance_ISS_Light1).T
distance_FERMI_Light1 = [haversine(Locations_data['Lon(deg)Light-1'][i],Locations_data['Lat(deg)Light-1'][i],Locations_data['Lon(deg)FERMI'][i],Locations_data['Lat(deg)FERMI'][i]) for i in range(len(Locations_data['Time(ModJDate)']))]
distance_FERMI_Light1 = np.array(distance_FERMI_Light1).T

# Add distances between sats to dict:
Locations_data.update({"ISS_Light-1_Distance(km)": distance_ISS_Light1})
Locations_data.update({"FERMI_Light-1_Distance(km)": distance_FERMI_Light1})

In [86]:
# Filter data line index and time by distance from Light-1:
THRESHOLD_km   = 1000     # Distance threshold (km)

ISS_idx_cut     = np.where([i <= THRESHOLD_km for i in distance_ISS_Light1])[0]
ISS_time_cut    = Locations_data['Time(ModJDate)'][ISS_idx_cut]

FERMI_idx_cut     = np.where([i <= THRESHOLD_km for i in distance_FERMI_Light1])[0]
FERMI_time_cut    = Locations_data['Time(ModJDate)'][FERMI_idx_cut]

# Format time in MJD and ISOT:
ISS_time = [Time(i,format="mjd") for i in ISS_time_cut]
ISS_time = [(i.mjd,i.isot) for i in ISS_time]

FERMI_time = [Time(i,format="mjd") for i in FERMI_time_cut]
FERMI_time = [(i.mjd,i.isot) for i in FERMI_time]

In [88]:
# In MJD: .00300 is 5 minutes.. so 0.01 for safety. 
# This is the threshold that distinguishes two satelite orbit overlaping durations from each other.
TIME_THRESHOLD = 0.01

num = 0 #for time pronted out: 0 for MJD, 1 for UTC
ISS_time_list = [('Start here:', ISS_time[0][num])]
ISS_val = 0 # To decide if correlation started at this time or ended at this time.
for i in range(len(ISS_time)-1):
    if ((ISS_time[i+1][0] - ISS_time[i][0]) >= TIME_THRESHOLD): #must stay in MJD for calculation

        if ((ISS_val % 2) == 0): # If val is even or odd
            k = ("End here:", ISS_time[i][num])
            
        else:
            k = ("Start here:", ISS_time[i+1][num])
        
        ISS_time_list.append(k)
        ISS_val = ISS_val + 1
k = ("End here:", ISS_time[-1][num])
ISS_time_list.append(k)


FERMI_time_list = [('Start here:', FERMI_time[0][num])]
FERMI_val = 0 # To decide if start or end
for i in range(len(FERMI_time)-1):
    if ((FERMI_time[i+1][0] - FERMI_time[i][0]) >= TIME_THRESHOLD):

        if ((FERMI_val % 2) == 0): # If val is even or odd
            k = ("End here:", FERMI_time[i][num])
            
        else:
            k = ("Start here:", FERMI_time[i+1][num])
        
        FERMI_time_list.append(k)
        FERMI_val = FERMI_val + 1
k = ("End here:", FERMI_time[-1][num])
FERMI_time_list.append(k)

In [47]:
# ISS_time_list
# FERMI_time_list

In [89]:
FERMI_Data_During_Light1_Mission = "../../Data/Other_Sat_Data/FERMI_all_Data_During_Light-1_Mission.csv"

FERMI_data_pd = pd.read_csv(FERMI_Data_During_Light1_Mission)
FERMI_Headers = list(FERMI_data_pd.keys())
FERMI_data    = [i for i in FERMI_data_pd.to_numpy().T]
FERMI_data    = dict(zip(FERMI_Headers,FERMI_data))

In [90]:
# FERMI_time = [Time(i,format="mjd") for i in FERMI_time_cut]
# FERMI_time = [(i.mjd,i.isot) for i in FERMI_time]
new_time_threshold = 0.003
for i in range(len(FERMI_time_list)-1):
    if 'Start' in FERMI_time_list[i][0] and 'End' in FERMI_time_list[i+1][0]:
        for k in range(len(FERMI_data['Trigger_Time(MJD)'])-1):
            # print (k)
            if  (FERMI_time_list[i][1] - new_time_threshold) < FERMI_data['Trigger_Time(MJD)'][k] < (FERMI_time_list[i+1][1] + new_time_threshold):
                
                Print_line = (FERMI_data['Trigger_Time(MJD)'][k],FERMI_data['Name'][k],FERMI_data['Trigger_Timescale(ms)'][k])
                print (Print_line)

# now add distance

(59892.6063671, 'SGR221109606 ', 16)


In [ ]:
# maybe corrilate FERMI data with ASIM later (LIKE WE HAVE IT!!)
# We have it now!

In [4]:
# 1
# New!
# ASIM Data:

# ASIM's time and location data of TGF detections:
Location_File_ASIM     = "../../Data/Other_Sat_Data/ASIM_TGF_during_Light-1 - Modified.csv"
Locations_ASIM_pd      = pd.read_csv(Location_File_ASIM)
Locations_ASIM_headers = list(Locations_ASIM_pd.keys())
Locations_ASIM_data    = [i for i in Locations_ASIM_pd.to_numpy().T]
Locations_ASIM_data    = dict(zip(Locations_ASIM_headers,Locations_ASIM_data))


In [5]:
# 2
# IMPORTANT!!
# RUN ONCE ONLY! 
# Then it comment out!
# Adding the space so time can be read by astropy.

# for i in range(len(Locations_ASIM_pd['####'])):
#     timeasim = Locations_ASIM_data['yyyy-MM-ddHH:mm:ss.SSSSSS'][i]
#     timeasim = timeasim[0:10]+' '+timeasim[10:] # fixing some issues
#     # print (timeasim)
#     Locations_ASIM_data['yyyy-MM-ddHH:mm:ss.SSSSSS'][i] = timeasim
# Locations_ASIM_data['yyyy-MM-ddHH:mm:ss.SSSSSS']

array(['2022-04-06 10:55:03.747481', '2022-04-06 16:25:16.880516',
       '2022-04-07 13:03:46.122768', '2022-04-11 21:31:12.294797',
       '2022-04-13 01:22:35.161010', '2022-04-13 11:27:57.522841',
       '2022-04-13 11:29:01.091045', '2022-04-14 11:30:11.402743',
       '2022-04-16 19:56:49.742407', '2022-04-16 19:56:51.052508',
       '2022-04-16 22:17:01.720787', '2022-04-17 04:28:45.779490',
       '2022-04-17 16:05:39.988726', '2022-04-18 15:15:10.343197',
       '2022-04-18 19:45:56.878735', '2022-04-19 02:50:13.216817',
       '2022-04-19 18:16:43.899827', '2022-04-20 21:22:15.579322',
       '2022-04-23 09:44:02.969030', '2022-04-23 18:15:25.252330',
       '2022-04-24 20:31:37.097618', '2022-04-27 16:36:54.711633',
       '2022-04-29 19:33:44.594455', '2022-05-02 12:42:31.156357',
       '2022-05-03 13:29:57.257710', '2022-05-04 14:59:22.639297',
       '2022-05-04 14:59:22.849948', '2022-05-04 15:00:07.976305',
       '2022-05-04 15:00:38.245823', '2022-05-05 13:30:00.4072

In [6]:
# 3
# Add ASIM time in MJD to list:

MJD_Time_ASIM = [Time(i,format="iso").mjd for i in Locations_ASIM_data['yyyy-MM-ddHH:mm:ss.SSSSSS']]
Locations_ASIM_data.update({"ASIM-Time-MJD": MJD_Time_ASIM })

In [7]:
# 4
# Checking if Light-1 was ON during the detection of a TGF by ASIM:

# Open light-1 operations log file (when it was ON and OFF):
Light1_log        = '../../Data/METADATA/orbit_meta.csv'
Light1_log_pd     = pd.read_csv(Light1_log)
Light1_log_header = list(Light1_log_pd.keys())
Light1_log_data   = [i for i in Light1_log_pd.to_numpy().T]
Light1_log_data   = dict(zip(Light1_log_header,Light1_log_data))

# Define a function to check if data in rows are of iso format to point out corrupted lines
def is_iso_format(date_str):
    iso_regex = re.compile(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}(\.\d+)?(Z|[+-]\d{2}:\d{2})?')
    return bool(iso_regex.match(date_str))

In [8]:
# Use is_iso_format function to sort useful data from not useful

Light1_log_status_MJD_list = [] # list of script start and end times in MJD (useful and corrupt)

for i in range(len(Light1_log_data['start_time'])):
    if is_iso_format(str(Light1_log_data['start_time'][i])) == True:
        Log_Start_Time_MJD = Time(Light1_log_data['start_time'][i],format='iso').mjd # turn that date data into MJD format
    else: 
        Log_Start_Time_MJD = 'Corrupt' # call it corrupt

    if is_iso_format(str(Light1_log_data['end_time'][i])) == True:
        Log_End_Time_MJD = Time(Light1_log_data['end_time'][i],format='iso').mjd
    else: 
        Log_End_Time_MJD = 'Corrupt'

    Light1_log_status_MJD = (i,Log_Start_Time_MJD,Log_End_Time_MJD)
    Light1_log_status_MJD_list.append(Light1_log_status_MJD)

In [9]:
# Arrays to be used for the calculations
Locations_ASIM_data_np = np.array(Locations_ASIM_data["ASIM-Time-MJD"])
Light1_log_status_MJD_list_np = np.array(Light1_log_status_MJD_list)

In [10]:
# Find the times where both ASIM detected a TGF and Light1 was ON

Light1_ON_ASIM_TGF_List = []

for ASIM_time in range(len(Locations_ASIM_data["ASIM-Time-MJD"])):
    for Light1_time in Light1_log_status_MJD_list:
        if not isinstance(Light1_time[1],str) and not isinstance(Light1_time[2],str):  # Ignore the corrupt lines which are in str format
            if Light1_time[1] <= Locations_ASIM_data["ASIM-Time-MJD"][ASIM_time] <=  Light1_time[2]: # If ASIM detected a TGF during the time of one of Light1's scripts where it was ON..
                line_to_print = (Locations_ASIM_data['####'][ASIM_time],Locations_ASIM_data['ASIM-Time-MJD'][ASIM_time],Locations_ASIM_data['Lat'][ASIM_time],Locations_ASIM_data['Lon'][ASIM_time],Light1_time[0],Light1_time[1],Light1_time[2])
                # print (line_to_print)
                Light1_ON_ASIM_TGF_List.append(line_to_print)

In [16]:
# Light1_ON_ASIM_TGF_List

In [19]:
# Printing out TGF time only (for viewing)
TGF_TIME_ONLY = [Time(i[1],format='mjd').iso for i in Light1_ON_ASIM_TGF_List]
TGF_TIME_ONLY

['2022-05-21 20:15:25.416',
 '2022-05-22 20:14:53.904',
 '2022-06-03 14:32:00.877',
 '2022-06-13 22:07:25.237',
 '2022-06-14 08:59:59.418',
 '2022-06-17 09:48:10.063',
 '2022-06-18 07:24:02.700',
 '2022-06-20 19:50:36.983',
 '2022-06-22 19:10:15.566',
 '2022-06-23 18:14:16.035',
 '2022-06-29 15:13:47.912',
 '2022-06-29 19:44:45.082',
 '2022-07-03 18:15:05.681',
 '2022-07-03 21:13:47.071',
 '2022-07-10 00:15:41.535',
 '2022-07-15 14:01:43.821',
 '2022-07-23 18:31:25.234',
 '2022-07-26 20:12:28.315',
 '2022-07-26 20:13:09.122',
 '2022-07-30 16:57:33.652',
 '2022-08-02 17:48:54.340',
 '2022-08-14 21:07:23.125',
 '2022-08-20 20:24:30.829',
 '2022-08-22 09:36:49.358',
 '2022-08-30 03:16:04.822',
 '2022-09-03 02:12:14.150',
 '2022-09-06 09:14:18.486',
 '2022-09-08 12:17:09.152',
 '2022-09-11 20:05:00.000',
 '2022-09-22 16:01:38.872',
 '2022-09-23 01:57:03.781',
 '2022-09-23 18:14:06.842',
 '2022-09-25 06:32:53.654',
 '2022-09-27 00:17:34.933',
 '2022-09-29 03:23:59.541',
 '2022-10-08 00:10:3

In [20]:
# times = [TGF_TIME_ONLY]

In [154]:
# test_asim_time = rp.Time(TGF_TIME_ONLY,format='mjd').T
test_asim_time = rp.Time([59744.37499326, 59747.40844981],format='mjd')
loc = rp.get_light1_position(test_asim_time)

DatabaseError: Execution failed on sql 'SELECT * FROM `LIGHT-1_Position_Data`.PositionData WHERE `Time (ModJDate)` BETWEEN [59744.37487752 59747.40833407] AND [59744.375109   59747.40856555];': (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '[59744.37487752 59747.40833407] AND [59744.375109   59747.40856555]' at line 1")

In [168]:
min_time    = rp.Time(min(TGF_TIME_ONLY),format='mjd')
max_time    = rp.Time(max(TGF_TIME_ONLY),format='mjd')
positions   = rp.get_light1_position(min_time,max_time,n_events=100000)

In [ ]:
# Interpolate
def interpolate(array,timestamps,format='mjd'):
    new_array   = rp.array([None]*len(timestamps))
    old_time    = array.get_timestamps(format=format)
    longs,lats  = array.get_coords().T
    for i,t in rp.tqdm(timestamps):
        new_array.append(rp.event(
            timestamp = rp.Time(t,format=format),
            latitude  = np.interp(t,old_time,lats),
            longitude = np.interp(t,old_time,longs),
            detector_id=0,
        ))


    return new_array

positions = interpolate(positions,TGF_TIME_ONLY)

array([[140.95126373, -38.80447916]])

In [ ]:
for i in range(len(Light1_ON_ASIM_TGF_List)):
     
    ASIM_TGF_lat    = Light1_ON_ASIM_TGF_List[i][2]
    ASIM_TGF_lon    = Light1_ON_ASIM_TGF_List[i][3]
    ASIM_MJD_time   = Light1_ON_ASIM_TGF_List[i][1]
    if ASIM_MJD_time

distance_ISS_Light1 = [haversine(Locations_data['Lon(deg)Light-1'][i],Locations_data['Lat(deg)Light-1'][i],Locations_data['Lon(deg)ISS'][i],Locations_data['Lat(deg)ISS'][i]) for i in range(len(Locations_data['Time(ModJDate)']))]
distance_ISS_Light1 = np.array(distance_ISS_Light1).T

In [ ]:
# # Distance between Light-1 and ASIM:
# distance_ASIM_Light1 = [haversine(Locations_data['Lon(deg)Light-1'][i],Locations_data['Lat(deg)Light-1'][i],Locations_ASIM_data['Lon'][i],Locations_ASIM_data['Lat'][i]) for i in range(len(Locations_ASIM_data['####']))]
# distance_ASIM_Light1 = np.array(distance_ASIM_Light1).T

# # Add distances between sats to dict:
# Locations_data.update({"ASIM_Light-1_Distance(km)": distance_ASIM_Light1})